In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, load_component, Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline
from promptflow import PFClient

import os
import time
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from azure.ai.ml.entities import Data, AmlCompute, BatchEndpoint, PipelineComponentBatchDeployment

In [7]:
# Load environment variables from .env file
load_dotenv(find_dotenv(), override=True)

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

print(os.getenv("WORKSPACE_NAME"))
print(os.getenv("AZURE_OPENAI_ENDPOINT"))

victor-aml
https://victor-aoai.openai.azure.com/


In [8]:
# Set config parameters

# PFClient can help manage your runs and connections.
pf = PFClient()

# Define Flows and Data
grey_customer_data = Input(path="../sample_data/sample_chat_outputs/grey_tests_customer.json", type='uri_file')
grey_grader_data = Input(path="../sample_data/sample_chat_outputs/grey_tests_grader.json", type='uri_file')
red_test_data = Input(path="../sample_data/sample_chat_outputs/red_tests.json", type='uri_file')

In [9]:
grey_cust_component = load_component("../src/components/evaluation_framework/promptflow/eval_flows/grey_eval_customer/flow.dag.yaml")
grey_grader_component = load_component("../src/components/evaluation_framework/promptflow/eval_flows/grey_eval_grader/flow.dag.yaml")
red_test_component = load_component("../src/components/evaluation_framework/promptflow/eval_flows/red_eval/flow.dag.yaml")

In [10]:
@pipeline
def pipeline_func_with_flow(grey_cust_input, grey_grader_input, red_test_input):
    grey_cust_flow = grey_cust_component(
        data=grey_cust_input,
        generated_question = "${grey_cust_input.generated_question}"
    )

    grey_grader_flow = grey_grader_component(
        data=grey_grader_input,
        generated_question = "${grey_grader_input.generated_question}",
        gt_response = "${grey_grader_input.ground_truth_response}",
        user_response = "${grey_grader_input.user_response}",
        evaluation = "${grey_grader_input.evaluation}",
        evaluation_score = "${grey_grader_input.evaluation_score}"
    )

    red_test_flow = red_test_component(
        data=red_test_input,
        user_input="${red_test_input.user_response}",
        chat_response="${red_test_input.chat_response}"
    )

pipeline_with_flow = pipeline_func_with_flow(grey_customer_data, grey_grader_data, red_test_data)
pipeline_with_flow.compute = "cpu-target"

pipeline_job = ml_client.jobs.create_or_update(pipeline_with_flow)
ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


RunId: gray_scooter_38frkt78xt
Web View: https://ml.azure.com/runs/gray_scooter_38frkt78xt?wsid=/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/victor-rg/workspaces/victor-aml

Streaming logs/azureml/executionlogs.txt

[2024-03-18 20:28:33Z] Submitting 3 runs, first five are: 6a62dd92:74baf96b-0f6f-4aa4-9878-2cdcd01fd46f,73fa00ff:a4d9f316-30d9-4241-916f-0e5917844c65,8391981c:ef246fa0-a541-4517-94e1-d59b46c1ba69
[2024-03-18 20:44:44Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: gray_scooter_38frkt78xt
Web View: https://ml.azure.com/runs/gray_scooter_38frkt78xt?wsid=/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/victor-rg/workspaces/victor-aml


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. Failed nodes: /grey_cust_flow, /red_test_flow, /grey_grader_flow. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "centralus",
    "location": "centralus",
    "time": "2024-03-18T20:44:44.207Z",
    "component_name": ""
} 